In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize,RegexpTokenizer
import nltk
from nltk.corpus import stopwords,wordnet
from sklearn.preprocessing import LabelEncoder
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

In [19]:
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [32]:
data=pd.read_csv("/content/drive/MyDrive/train_2kmZucJ.csv")
data

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...
...,...,...,...
7915,7916,0,Live out loud #lol #liveoutloud #selfie #smile...
7916,7917,0,We would like to wish you an amazing day! Make...
7917,7918,0,Helping my lovely 90 year old neighbor with he...
7918,7919,0,Finally got my #smart #pocket #wifi stay conne...


In [31]:
for i in range(len(data['review'])):
     data['review'][i]=word_tokenize(data['review'][i])

KeyError: ignored

### stopword removal

In [33]:
re=RegexpTokenizer('[a-zA-Z]+')


In [34]:
data['tweet']=[re.tokenize(sent) for sent in data['tweet']]

In [80]:
def clean_data(data):
    data=[re.tokenize(sent) for sent in data]
    data=[remove_stopwords(sent) for sent in data]
    return data

In [35]:
sw=list(stopwords.words('english'))
sw.append('http')
sw.append('https')
sw.append('com')


In [38]:
def remove_stopwords(text):
  new_text=[word for word in text if word.lower() not in sw and len(word)>1]
  return new_text

In [39]:
data['tweet']=[remove_stopwords(sent) for sent in data['tweet']]

In [50]:
data=data.drop(columns=['id'])

In [51]:
data

,label,tweet
0,0,"[fingerprint, Pregnancy, Test, goo, gl, MfQV, ..."
1,0,"[Finally, transparant, silicon, case, Thanks, ..."
2,0,"[love, Would, go, talk, makememories, unplug, ..."
3,0,"[wired, know, George, make, way, iphone, cute,..."
4,1,"[amazing, service, Apple, even, talk, question..."
...,...,...
7915,0,"[Live, loud, lol, liveoutloud, selfie, smile, ..."
7916,0,"[would, like, wish, amazing, day, Make, every,..."
7917,0,"[Helping, lovely, year, old, neighbor, iPad, m..."
7918,0,"[Finally, got, smart, pocket, wifi, stay, conn..."


# Lemmatization

In [15]:
lemmatizer=WordNetLemmatizer()

In [16]:
def get_pos_tag(tag):
    if tag.startswith('J'):
      return wordnet.ADJ
    elif tag.startswith('V'):
         return wordnet.VERB 
    elif tag.startswith('N'):
         return wordnet.NOUN 
    elif tag.startswith('R'):
         return wordnet.ADV
    else:
         return wordnet.NOUN                 

In [17]:
def clean_text(text):
  output=[]
  for w in text:
    pos=pos_tag(w)
    clean=lemmatizer.lemmatize(w,get_pos_tag(pos[0][1]))
    output.append(clean)
  return output  
  

In [41]:
data['tweet']=[clean_text(sent) for sent in data['tweet']]

# TEXT TO FEATURES

In [ ]:
for i  in range(len(data['tweet'])):
  lists=[]
  for w in data['tweet'][i]:
    lists.append(w);
  line=" ".join(lists)
  data.at['tweet',i]=line

MemoryError: ignored

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv=CountVectorizer()

In [ ]:
vectorized=cv.fit_transform(data['tweet'])

In [ ]:
vectorized

<7920x22015 sparse matrix of type '<class 'numpy.int64'>'
	with 105363 stored elements in Compressed Sparse Row format>

# Bag of Words

In [21]:
def all_words(data):
  bag_of_words=[]
  for sent in data:
    for w in sent:
      bag_of_words.append(w)
  return bag_of_words    

In [42]:
clean_words=all_words(data['tweet'])

In [43]:
len(clean_words)

108933

In [44]:
count=nltk.FreqDist(clean_words)

In [47]:
features=count.most_common(100)

In [48]:
feature=[i[0] for i in features]

In [64]:
def get_feature_dict(data):
  current_features={}
  word_set=set(data)
  for w in feature:
    current_features[w]=w in word_set
  return list(current_features.values())  

In [65]:
training_data=[get_feature_dict(text) for text in data['tweet']]

In [69]:
y_train=data['label'].values

In [71]:
from sklearn.naive_bayes import GaussianNB

In [72]:
clf=GaussianNB()

In [73]:
clf.fit(training_data,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [74]:
clf.score(training_data,y_train)

0.6059343434343434

In [75]:
test_data=pd.read_csv('/content/drive/MyDrive/test_oJQbWVk.csv')

In [83]:
ids=test_data['id']

In [81]:
test_data['tweet']=clean_data(test_data['tweet'])

In [84]:
testing_data=[get_feature_dict(text) for text in test_data['tweet']]

In [86]:
preds=clf.predict(testing_data)

In [93]:
np.count_nonzero(preds)

1341

In [91]:
import numpy as np


In [106]:
df=pd.DataFrame(data=ids,columns=['id'])

In [96]:
df['id']=ids

In [107]:
df['label']=preds

In [109]:
file='AVanswer.csv'
fg=df.to_csv(file,index=False)

In [110]:
df

,id,label
0,7921,1
1,7922,1
2,7923,1
3,7924,1
4,7925,1
...,...,...
1948,9869,1
1949,9870,0
1950,9871,1
1951,9872,1
